In [1]:
print('hi')

hi


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [3]:
# Set parameters
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True
img_width, img_height = 224, 224
batch_size = 32
epochs = 20
num_classes = 47
learning_rate = 0.0001

# Define the data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(img_width),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    #,
    #'val': transforms.Compose([
     #   transforms.Resize(256),
     #   transforms.CenterCrop(img_width),
     #   transforms.ToTensor(),
     #   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    #]),
}


In [4]:
# Define the dataset directories
train_data_dir = '/newstorage5/aayesha/deep_learning/classification/pattern_classification/pattern_dataset'
#validation_data_dir = '/Users/aayeshaaayesha/Downloads/dtd/root/train'

# Load the datasets
train_dataset = datasets.ImageFolder(train_data_dir, transform=data_transforms['train'])
#val_dataset = datasets.ImageFolder(validation_data_dir, transform=data_transforms['val'])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Load the ResNet50 model pre-trained on ImageNet
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer to match the number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)



/newstorage5/aayesha/venv/imgenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/newstorage5/aayesha/venv/imgenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Training loop
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    #for phase in ['train', 'val']:
    for phase in ['train']:
        if phase == 'train':
            model.train()  # Set model to training mode
            dataloader = train_loader
        #else:
         #   model.eval()  # Set model to evaluate mode
          #  dataloader = val_loader

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward pass and optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    print()

Epoch 1/20
----------
train Loss: 2.0548 Acc: 0.4768

Epoch 2/20
----------
train Loss: 1.2351 Acc: 0.6438

Epoch 3/20
----------
train Loss: 1.0232 Acc: 0.6968

Epoch 4/20
----------
train Loss: 0.8764 Acc: 0.7378

Epoch 5/20
----------
train Loss: 0.7550 Acc: 0.7713

Epoch 6/20
----------
train Loss: 0.7080 Acc: 0.7817

Epoch 7/20
----------
train Loss: 0.6407 Acc: 0.7998

Epoch 8/20
----------
train Loss: 0.5723 Acc: 0.8225

Epoch 9/20
----------
train Loss: 0.5169 Acc: 0.8383

Epoch 10/20
----------
train Loss: 0.4798 Acc: 0.8537

Epoch 11/20
----------
train Loss: 0.4592 Acc: 0.8535

Epoch 12/20
----------
train Loss: 0.4377 Acc: 0.8601

Epoch 13/20
----------
train Loss: 0.3788 Acc: 0.8869

Epoch 14/20
----------
train Loss: 0.3824 Acc: 0.8824

Epoch 15/20
----------
train Loss: 0.3381 Acc: 0.8973

Epoch 16/20
----------
train Loss: 0.3474 Acc: 0.8952

Epoch 17/20
----------
train Loss: 0.3477 Acc: 0.8895

Epoch 18/20
----------
train Loss: 0.3242 Acc: 0.8968

Epoch 19/20
-------

In [47]:
print(labels)

tensor([ 4, 15, 41,  2,  7, 44, 23, 33], device='cuda:0')


In [ ]:
# Save the trained model
#torch.save(model.state_dict(), 'resnet50_custom_pattern_multiclass_model_20Epochs.pth')

In [1]:

from PIL import Image

# Check if GPU is available and use it, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the ResNet-50 model and move it to the appropriate device
#model = models.resnet50(pretrained=True)
model = model.to(device)  # Move the mode
model.eval()  # Set the model to evaluation mode

# Define the image transformations (e.g., resize, normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to the expected input size for ResNet-50
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Load the image
image = Image.open('/newstorage5/aayesha/red_zigzag.jpg')  # Replace 'input_image.jpg' with your image file path
image = transform(image)  # Apply the transformations
image = image.unsqueeze(0).to(device)  # Add a batch dimension and move to the appropriate device

# Perform inference
with torch.no_grad():
    output = model(image)  # Pass the image through the model

# Interpret the output
_, predicted = torch.max(output, 1)
print(predicted)
print(f'Predicted class: {predicted.item()}')
x = predicted.item()
print(x)


NameError: name 'torch' is not defined

In [56]:
dir_path = "/newstorage5/aayesha/deep_learning/classification/pattern_classification/pattern_dataset"
import os
i = -2
sorted_items = sorted(os.listdir(dir_path))
for name in sorted_items:
    #print(dir_path+"/"+name)
    i = i + 1
    if i == x:
        y = dir_path+"/"+name
        print(name)


studded
